In [72]:
# Amend to set wd to STB_social_media_analytics.
%cd /home/zyf0717/git_environment/STB_social_media_analytics/

/home/zyf0717/git_environment/STB_social_media_analytics


In [73]:
import pandas as pd
import mysql.connector
import re
import yaml
import utils
from random import random
from selenium import webdriver
from datetime import datetime
from time import sleep

In [44]:
with open('config_file.yml') as file:
    configs = yaml.load(file, Loader=yaml.FullLoader)

chromedriver_path = configs['General']['chromedriver_path']

### Change poi_df as necessary ###
pois = ['Gardens by the Bay', 'Marina Bay Sands Skypark']
urls = ['https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-Gardens_by_the_Bay-Singapore.html',
         'https://www.tripadvisor.com.sg/Attraction_Review-g294265-d1837767-Reviews-Marina_Bay_Sands_Skypark-Singapore.html']
poi_df = pd.DataFrame({'poi':pois, 'urls':urls})
##################################

db_in_flag = configs['TripAdvisor']['db_in_flag']
db_out_flag = configs['TripAdvisor']['db_out_flag']

if db_in_flag == True:
    poi_df = pd.DataFrame()

if db_in_flag == True or db_out_flag == True:
    cnx = mysql.connector.connect(host=configs['General']['host'],
                                  database=configs['General']['database'],
                                  user=configs['General']['user'],
                                  password=configs['General']['password'])
else:
    cnx = None

poi_df

,poi,urls
0,Gardens by the Bay,https://www.tripadvisor.com.sg/Attraction_Revi...
1,Marina Bay Sands Skypark,https://www.tripadvisor.com.sg/Attraction_Revi...


In [70]:
class CrawlTripAdvisor:
    def __init__(self, chromedriver_path, poi_df, cnx, db_out_flag):
        if cnx is not None:
            self.cursor = cnx.cursor()
        self.poi_df = poi_df
        self.reviews = []
        self.driver = webdriver.Chrome(chromedriver_path) 
        

    def add_attributes_to_database(self, attributes):
        return
    
    
    def add_reviews_to_database(self, reviews):
        return
    
    
    def crawl_pois(self, number_of_pages=None, earliest_date=None):
        for index, row in poi_df.iterrows():
            self.driver.get(row['urls'])
            self.crawl_attributes()
            self.crawl_reviews(number_of_pages, earliest_date)
    
    
    def crawl_reviews(self, number_of_pages=None, earliest_date=None):
        if earliest_date == None:
            for i in range(number_of_pages):
                self.crawl_reviews_1_page()
        else:
            pass

    
    def crawl_attributes(self):
        driver = self.driver
        
        total_reviews_text = driver.find_element_by_xpath('//span[@class="reviewCount"]').text
        ranking_text = driver.find_element_by_xpath('//span[@class="header_popularity popIndexValidation "]').text
        rating_text = driver.find_element_by_xpath('//span[@class="overallRating"]').text
        rating_proportion_elements = driver.find_elements_by_xpath('//span[@class="row_count row_cell"]')
        address_text_1 = driver.find_element_by_xpath('//span[@class="street-address"]').text
        address_text_2 = driver.find_element_by_xpath('//span[@class="extended-address"]').text
        address_text_3 = driver.find_element_by_xpath('//span[@class="locality"]').text
        address_text_4 = driver.find_element_by_xpath('//span[@class="country-name"]').text

        about_more_button = driver.find_elements_by_xpath('//span[@class="attractions-attraction-detail-about-card-Description__readMore--2pd33"]')
        if about_more_button != []:
            about_more_button[0].click()
            sleep(0.5)
            about_text = driver.find_element_by_xpath('//div[@class="attractions-attraction-detail-about-card-Description__modalText--1oJCY"]').text
            about_more_close_button = driver.find_element_by_xpath('//div[@class="_2EFRp_bb"]')
            about_more_close_button.click()
            sleep(0.5)
        else:
            about_text = driver.find_element_by_xpath('//div[@class="attractions-attraction-detail-about-card-AttractionDetailAboutCard__section--1_Efg"]').text

        attributes_crawled_time = datetime.now()
        
        # insert into pd df -> csv / sql
        print(self.parse_total_reviews_text(total_reviews_text))
        print(self.parse_ranking_text(ranking_text))
        print(self.parse_rating_text(rating_text))
        print(self.parse_rating_proportion_elements(rating_proportion_elements))
        print(about_text)
        print(self.parse_address_text(address_text_1, address_text_2, address_text_3, address_text_4))
        print(attributes_crawled_time)
        print()
    

    def crawl_reviews_1_page(self):        
        driver = self.driver
        
        # while current page != last page:
        read_more_button = driver.find_element_by_xpath('//span[@class="location-review-review-list-parts-ExpandableReview__cta--2mR2g"]')
        read_more_button.click()
        sleep(1)

        reviewer_url_elements = driver.find_elements_by_xpath('//a[@class="ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC"]')
        reviewer_details_elements = driver.find_elements_by_xpath('//div[@class="social-member-event-MemberEventOnObjectBlock__event_wrap--1YkeG"]')
        review_rating_elements = driver.find_elements_by_xpath('//div[@class="location-review-review-list-parts-RatingLine__bubbles--GcJvM"]/span')
        review_title_elements = driver.find_elements_by_xpath('//a[@class="location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT"]')
        review_body_elements = driver.find_elements_by_xpath('//div[@class="location-review-review-list-parts-ExpandableReview__containerStyles--1G0AE"]')
        date_of_experience_elements = driver.find_elements_by_xpath('//span[@class="location-review-review-list-parts-EventDate__event_date--1epHa"]')
        reviewer_name_elements = reviewer_url_elements
        review_crawled_time = datetime.now()
        
        # insert into pd df -> csv / sql
        for i in range(len(reviewer_url_elements)):
            print(reviewer_url_elements[i].get_attribute('href'))
            print(reviewer_name_elements[i].text)
            print(self.parse_review_date(reviewer_details_elements[i].text))
            print(self.parse_location_contributions_votes(reviewer_details_elements[i].text))
            print(self.parse_review_rating(review_rating_elements[i].get_attribute('class')))
            print(review_title_elements[i].text)
            print(self.parse_review_body(review_body_elements[i].text))
            print(self.parse_date_of_experience(review_body_elements[i].text))
            print(self.parse_trip_type(review_body_elements[i].text))
            print()
            
        next_button = driver.find_element_by_xpath('//a[@class="ui_button nav next primary "]')
        if next_button != []:
            next_button.click()
            sleep(1)
    
    
    def parse_total_reviews_text(self, text):
        return int(text[:text.find(' Reviews')].replace(',', ''))
    
    
    def parse_ranking_text(self, text):
        return int(text[1:text.find(' of')].replace(',', ''))
    
    
    def parse_rating_text(self, text):
        return float(text)
    
    
    def parse_rating_proportion_elements(self, elements):
        rating_proportion = []
        for element in elements:
            if element.text != None:
                text = element.text
                rating_proportion.append(float(text[:text.find('%')])/100)
            else:
                rating_proportion.append(None)
        return rating_proportion
        
    
    def parse_address_text(self, text_1, text_2, text_3, text_4):
        return ('{}, {}, {} {}'.format(text_1, text_2, text_3, text_4))
    
    
    def parse_review_date(self, text):
        return text[text.find('wrote a review ')+15:text.find('\n')]

    
    def parse_location_contributions_votes(self, text):
        try:
            votes = int(re.search('(\d+) helpful votes?', text).group(1))
        except:
            votes = None

        try:
            contributions = int(re.search('(\d+) contributions?', text).group(1))
        except:
            contributions = None

        try:
            location = re.search('(.+?){} contributions?'.format(contributions), text).group(1)
        except:
            location = None

        return location, contributions, votes

    
    def parse_review_rating(self, text):
        return int(text[-2:])//10

    
    def parse_review_body(self, text):
        return text[:text.find('Read less')-1]

    
    def parse_date_of_experience(self, text):
        substring = text[text.find('Date of experience: ')+20:]
        return substring[:substring.find('\n')]

    
    def parse_trip_type(self, text):
        if text.find('Trip type: ') == -1:
            return None
        substring = text[text.find('Trip type: ')+11:]
        return substring[:substring.find('\n')]
    

In [71]:
CrawlTripAdvisor(chromedriver_path, poi_df, cnx, db_out_flag).crawl_pois(number_of_pages=2)

58483
3
4.5
[0.71, 0.23, 0.04, 0.01, 0.01]
An integral part of Singapore's "City in a Garden" vision, Gardens by the Bay spans a total of 101 hectares of prime land at the heart of Singapore's new downtown - Marina Bay. Comprising three waterfront gardens - Bay South, Bay East and Bay Central - Gardens by the Bay will be a showcase of horticulture and garden artistry that will bring the world of plants to Singapore and present Singapore to the World.
18 Marina Gardens Drive, Bayfront Plaza, Singapore 018953, Singapore
2020-01-15 15:22:05.173726

https://www.tripadvisor.com.sg/Profile/Twisterella11
Twisterella11
Yesterday
('Tamworth, Australia', 82, 22)
5
GBTB is Singapore to a T
Beautiful clean gardens, we have been to the flower dome, floral fantasy, super tree grove and Cloud forest and would recommend them all!!
January 2020
Travelled as a couple

https://www.tripadvisor.com.sg/Profile/colsktm
CP_NZ
Yesterday
('Palmerston North, New Zealand', 9, 4)
4
Family Holiday
Amazing Building,

In [87]:
class MyClass:
    def __init__(self):
        self.name = "Yifei"
        self.age = 27
    
    def add_square_age(self, x):
        y = self.square(x)
        self.age += y
        
    def square(self, x):
        return x**2
    
    def root(x):
        return x**0.5

In [88]:
a = MyClass()
a.age

27

In [89]:
a.add_square_age(2)
a.age

31

In [90]:
MyClass.root(4)

2.0